In [ ]:
import numpy as np
from tqdm import tqdm
import torch
from torch import nn, optim
from torch.optim.lr_scheduler import StepLR
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, random_split, Dataset
import torch.nn.functional as F

Q = np.genfromtxt('./data/stiffness.csv', delimiter=',')
A = np.genfromtxt('./data/volumefraction.csv', delimiter=',')
lamda = ((np.ones((6,6))+np.eye(6))/2).reshape(-1)

In [ ]:
init_ODF = np.loadtxt("./data/init_ODF.txt")

X = torch.Tensor(init_ODF[:5000])
Y = np.zeros((5000, 31, 145))
for i in range(31):
    Y[:, i, :] = np.loadtxt(f"./data/train_data/simu{i}_result.txt")
Y = torch.Tensor(Y)

In [ ]:
class ODFDataset(Dataset):
    def __init__(self, x, y, transform=None):
        self.x = x
        self.y = y
        self.transform = transform
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        sample = (self.x[idx], self.y[idx])
        return sample

In [ ]:
class NN(nn.Module):
    def __init__(self, A):
        super().__init__()
        self.dim_list = [76, 760, 76]
        self.num_layers = len(self.dim_list)
        self.weights = nn.ParameterList([nn.Parameter(torch.empty(31, self.dim_list[i], self.dim_list[i+1])) for i in range(self.num_layers-1)])
        self.bias = nn.ParameterList([nn.Parameter(torch.empty(31, 1, self.dim_list[i+1])) for i in range(self.num_layers-1)])
        self.activations = [nn.Tanh()]*(self.num_layers-2) + [nn.ReLU()]
        
        self.init_weights()
        self.A = torch.Tensor(A).cuda()
        
    def init_weights(self):
        for i in range(self.num_layers-1):
            nn.init.xavier_normal_(self.weights[i])
            self.bias[i].data.zero_()
        
    def forward(self, x: torch.Tensor):
        y = x[:, None, None, :]
        for i in range(self.num_layers-1):
            y = y@self.weights[i]+self.bias[i]
            y = self.activations[i](y)
        pred_y = (y.squeeze())/(y@self.A)

        return pred_y

In [ ]:
torch.manual_seed(3407)
x = X[:,:76]
y = Y[:,:,:76]

model = NN(A).cuda()
criterion = nn.MSELoss()

dst = ODFDataset(x, y)

def seed_worker(worker_id):
    worker_seed = torch.initial_seed() % 2**32
    numpy.random.seed(worker_seed)
    random.seed(worker_seed)

g = torch.Generator()
g.manual_seed(0)

train_dst, val_dst = random_split(dst, [4000, 1000])
train_loader = DataLoader(train_dst, 128, shuffle=True)
val_loader = DataLoader(val_dst, 1000, worker_init_fn=seed_worker, generator=g, shuffle=False)

IPT = torch.Tensor(lamda@Q/(min(lamda@Q))).cuda()
LOSS = np.empty(0)
VAL_LOSS = np.empty(0)
VAL_stiferr = []
opt = [
       (optim.Adam, {"lr": 5e-5}, 100),
       (optim.SGD, {"lr": 1e-4, "momentum":.9}, 200),
       (optim.Adam, {"lr": 2e-5}, 1000),
       (optim.Adam, {"lr": 1e-4}, 500),
       (optim.Adam, {"lr": 2e-5}, 1000),
       (optim.Adam, {"lr": 1e-4}, 500),
       (optim.Adam, {"lr": 2e-5}, 1000),
       ]
for i in range(len(opt)):
    optimizer = opt[i][0](model.parameters(), **opt[i][1])
    for e in tqdm(range(opt[i][2])):
        training_loss = 0
        for p, q in train_loader:
            p = p.cuda()
            q = q.cuda()
            optimizer.zero_grad()
            pred_q = model(p)
            loss = criterion(q*IPT, pred_q*IPT)
            training_loss += loss
            loss.backward()
            optimizer.step()
        LOSS = np.append(LOSS, training_loss.detach().cpu().numpy())
        print(f"\nEPOCH {e}: LOSS={training_loss}")
      # print("START VALIDATION")
      with torch.no_grad():
            for p, q in val_loader:
                p = p.cuda()
                q = q.cuda()
                pred_q = model(p)
                val_loss = criterion(q*IPT, pred_q*IPT)
                print(f"LOSS={val_loss}")
                stiff_err = np.mean(abs(pred_q.cpu().numpy()@(lamda@Q)-q.cpu().numpy()@(lamda@Q)), axis=0)
                # print(f"AVG STIFNESS DIFF:{stiff_err}")
                VAL_LOSS = np.append(VAL_LOSS, val_loss.detach().cpu().numpy())
                VAL_stiferr.append(stiff_err)

In [ ]:
from numpy.random import multinomial

# Searing algorithm
def exp_weight_algo(input):
    HASH = {}
    with torch.no_grad():
    for _ in range(1000):
        path = []
        ipt = input
        for i in range(10):
            ipt = torch.Tensor(ipt).cuda()
            opt = model(ipt).cpu().numpy()
            stf = ((opt-ipt.cpu().numpy())@(lamda@Q))[0,:]
            if (stf>0).sum()==0:
                continue
            weights = np.exp(5*stf)
            weights = weights/weights.sum()
            idx = np.where(multinomial(1, weights) == 1)[0][0]
        
            ipt = opt[:,idx,:]
            path.append(idx)
        f_stf = opt[0,idx,:]@(lamda@Q)
        HASH[f_stf] = path
    return HASH